In [1]:
import torch
from src.data.argoverse_datamodule import ArgoverseDataModule
from src.data.argoverse_dataset import ArgoverseDataset
from torch.utils.data import DataLoader, Dataset

import os
os.environ["CUDA_VISIBLE_DEVICES"]= "2"

In [5]:
args = {"IFC":True, "add_centerline":False, "attention_heads":4, "batch_norm":False, "batch_size":2, "check_val_every_n_epoch":3, 
          "dataroot":'./data/argoverse_with_LRP', "dataset":'argoverse', "distributed_backend":'ddp', "dropout":0.0, 
          "early_stop_threshold":5, "experiment_name":'example', "gpus":3, "gradient_clipping":True, "graph_iter":1, 
          "hidden_dim":512, "hidden_key_generator":True, "hidden_transform":False, "input_dim":2, "k_value_threshold":10, 
          "k_values":[6, 5, 4, 3, 2, 1], "lr":0.0001, "map_features":False, "max_epochs":200, "mode":'train', "model_name":'WIMP', 
          "no_heuristic":False, "non_linearity":'relu', "num_layers":4, "num_mixtures":6, "num_nodes":1, "output_conv":True, "output_dim":2, 
          "output_prediction":True, "precision":32, "predict_delta":False, "resume_from_checkpoint":None, 
          "scheduler_step_size":[60, 90, 120, 150, 180], "seed":None, "segment_CL":False, "segment_CL_Encoder":False, 
          "segment_CL_Encoder_Gaussian":False, "segment_CL_Encoder_Gaussian_Prob":False, "segment_CL_Encoder_Prob":True, 
          "segment_CL_Gaussian_Prob":False, "segment_CL_Prob":False, "use_centerline_features":True, "use_oracle":False, "waypoint_step":5, 
          "weight_decay":0.0, "workers":8, "wta":False, "draw_image" : False, "remove_high_related_score" : False, "maximum_delete_num" : 3, 
          "save_json": False, "make_submit_file" : False, "use_hidden_feature" : True, "is_LRP": False, "adjacency_exp" : True}

from argparse import ArgumentParser
parser = ArgumentParser()


for k in args:
    parser.add_argument(str("--" + k), default = args[k], type= type(args[k]))
parser.add_argument("--XAI_lambda", default = 0.2, type= float)
parser.add_argument("--name", default = "", type=str)

try:  # terminal
    parser = parser.parse_args()
except:  # Jupyter notebook
    parser = parser.parse_args(args=[])



usage: ipykernel_launcher.py [-h] [--IFC IFC]
                             [--add_centerline ADD_CENTERLINE]
                             [--attention_heads ATTENTION_HEADS]
                             [--batch_norm BATCH_NORM]
                             [--batch_size BATCH_SIZE]
                             [--check_val_every_n_epoch CHECK_VAL_EVERY_N_EPOCH]
                             [--dataroot DATAROOT] [--dataset DATASET]
                             [--distributed_backend DISTRIBUTED_BACKEND]
                             [--dropout DROPOUT]
                             [--early_stop_threshold EARLY_STOP_THRESHOLD]
                             [--experiment_name EXPERIMENT_NAME] [--gpus GPUS]
                             [--gradient_clipping GRADIENT_CLIPPING]
                             [--graph_iter GRAPH_ITER]
                             [--hidden_dim HIDDEN_DIM]
                             [--hidden_key_generator HIDDEN_KEY_GENERATOR]
                             [--hi

In [9]:
from src.models.WIMP import WIMP
from tqdm import tqdm
val_loader = ArgoverseDataset(parser.dataroot, mode='val', delta=parser.predict_delta,
                              map_features_flag=parser.map_features,
                              social_features_flag=True, heuristic=(not parser.no_heuristic),
                              ifc=parser.IFC, is_oracle=parser.use_oracle)
val_dataset = DataLoader(val_loader, batch_size=parser.batch_size, num_workers=parser.workers,
                                pin_memory=True, collate_fn=ArgoverseDataset.collate,
                                shuffle=False, drop_last=False)
model = WIMP(parser)
model.load_state_dict(torch.load("experiments/example_old/checkpoints/epoch=122.ckpt")['state_dict'], strict=False) # 학습할 때에는 graph 모듈에서 p에 해당하는 network가 없었으므로

model = model.cuda()

for batch_idx, batch in enumerate(tqdm(val_dataset)):
    input_dict, target_dict = batch[0], batch[1]
    # get cuda
    input_dict["agent_features"] = input_dict["agent_features"].cuda()
    input_dict["social_features"] = input_dict["social_features"].cuda()
    input_dict["social_label_features"] = input_dict["social_label_features"].cuda()
    input_dict["adjacency"] = input_dict["adjacency"].cuda()
    input_dict["label_adjacency"] = input_dict["label_adjacency"].cuda()
    input_dict["num_agent_mask"] = input_dict["num_agent_mask"].cuda()
    input_dict["ifc_helpers"]["agent_oracle_centerline"] = input_dict["ifc_helpers"]["agent_oracle_centerline"].cuda()
    input_dict["ifc_helpers"]["agent_oracle_centerline_lengths"] = input_dict["ifc_helpers"]["agent_oracle_centerline_lengths"].cuda()
    input_dict["ifc_helpers"]["social_oracle_centerline"] = input_dict["ifc_helpers"]["social_oracle_centerline"].cuda()
    input_dict["ifc_helpers"]["social_oracle_centerline_lengths"] = input_dict["ifc_helpers"]["social_oracle_centerline_lengths"].cuda()
    input_dict["ifc_helpers"]["agent_oracle_centerline"] = input_dict["ifc_helpers"]["agent_oracle_centerline"].cuda()

    target_dict["agent_labels"] = target_dict["agent_labels"].cuda()

    break


  0%|          | 0/19736 [00:01<?, ?it/s]


In [10]:
def func(a,b,c,d):
    with torch.backends.cudnn.flags(enabled=False):
        preds, waypoint_preds, all_dist_params, attention, adjacency, gan_features, graph_output = model(a, b, c, d, ifc_helpers = input_dict["ifc_helpers"])
        loss, (ade, fde, mr) = model.eval_preds(preds, target_dict, waypoint_preds)
        loss = loss.cpu()
        print(loss)
        return loss
    

torch.autograd.functional.hessian(func,tuple([input_dict["agent_features"], input_dict["social_features"], input_dict["adjacency"],
                                                                                   input_dict["num_agent_mask"]]))
        

tensor(13.0451, grad_fn=<CopyBackwards>)


((tensor([[[[[[ 1.0386e+00, -1.2403e+00],
              [-5.1156e-01, -8.0698e-01],
              [ 8.2879e-01, -2.3238e-01],
              ...,
              [-4.3375e-01,  6.9592e-01],
              [-1.0057e+00, -2.1424e-01],
              [ 8.5640e-01, -8.9781e-01]],
  
             [[ 0.0000e+00,  0.0000e+00],
              [ 0.0000e+00,  0.0000e+00],
              [ 0.0000e+00,  0.0000e+00],
              ...,
              [ 0.0000e+00,  0.0000e+00],
              [ 0.0000e+00,  0.0000e+00],
              [ 0.0000e+00,  0.0000e+00]]],
  
  
            [[[-1.2372e+00,  2.6570e+00],
              [ 1.0952e-01,  2.1442e+00],
              [-1.6201e+00,  6.1181e-01],
              ...,
              [ 2.6072e-01, -1.3888e+00],
              [ 8.8744e-02,  1.7387e+00],
              [-1.0901e+00,  3.7524e+00]],
  
             [[ 0.0000e+00,  0.0000e+00],
              [ 0.0000e+00,  0.0000e+00],
              [ 0.0000e+00,  0.0000e+00],
              ...,
              [ 0.0000e+00

In [12]:
import time

start_time = time.time()
hs = torch.autograd.functional.hessian(func,tuple([input_dict["agent_features"], input_dict["social_features"], input_dict["adjacency"],
                                                                                   input_dict["num_agent_mask"]]))

end_time = time.time()

print(end_time - start_time)

tensor(13.0451, grad_fn=<CopyBackwards>)
1191.4800791740417


In [28]:
hs[0][0].shape

torch.Size([2, 20, 2, 2, 20, 2])